In [ ]:
import clips
import sys
sys.path.append('../../src/')
from clips_util import print_facts, print_rules, print_templates

# Logical Conditional Element

In this example, we'll see the effects of using the _logical conditional element_ (LCE) on fact maintenance. Briefly, facts generated on the right hand side of defrules in which portions of the left hand side are surrounded by `logical` will be removed if the truth (aka support) of those portions is changed.

In [ ]:
# create the CLIPS environment
env = clips.Environment()

In [ ]:
env.clear()

# Create patient template
DEFTEMPLATE_PATIENT = """
(deftemplate patient
   (slot abdominal_pain)
   (slot nausea)
   (slot vomiting)
   (slot white_blood_count (type INTEGER)))
"""

# Create action template
DEFTEMPLATE_ACTION = """
(deftemplate action
   (slot disposition))

"""

env.build(DEFTEMPLATE_PATIENT)
env.build(DEFTEMPLATE_ACTION)

# create a patient fact
patient_template = env.find_template('patient')
patient = patient_template.assert_fact(abdominal_pain = 'yes', 
                                       nausea = 'yes',
                                       vomiting = 'yes')

############ defrule without logical conditional element
DEFRULE_DIAGNOSE = """
(defrule diagnose
   (patient (abdominal_pain "yes")(nausea "yes")(vomiting "yes"))
 =>
   (println "suspect appendicitis")
   (assert(action(disposition "admit"))))
"""

env.build(DEFRULE_DIAGNOSE)
####################################################

############# defrule with logical
## To try this version, comment out the version above and uncomment the version below including the admit rule
## Notice the logical conditional element in the diagnose rule
## This will chain the support (patient) to the new fact (action)
## Now, if the patient fact is retracted, the action fact will also be retracted

# DEFRULE_DIAGNOSE = """
# (defrule diagnose
#    (logical (patient (abdominal_pain "yes")(nausea "yes")(vomiting "yes")))
#  =>
#    (println "suspect appendicitis" )
#    (assert(action(disposition "admit"))))
# """

# DEFRULE_ADMIT = """
# (defrule admit
#    (action(disposition "admit"))
#  =>
#    (println "find this patient a room!!!"))
# """

# env.build(DEFRULE_DIAGNOSE)
# env.build(DEFRULE_ADMIT)
####################################################

env.run()

In [ ]:
# with either version of the defrules above, there will be two facts 
print_facts(env)

In [ ]:
# retract the patient fact
# using the first version of the defules, the action fact will remain
# using the second version of the defules, the action fact will be automatically removed b/c the patient support has been
# retracted
patient.retract()
print_facts(env)